In [9]:
from pathlib import Path 
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

In [11]:
VERSION = "v1.1s"
VARIANT = "nocomments"
holdout_messages_df = pd.read_json(f"SFT_message_format_{VARIANT}_{VERSION}.jsonl", lines=True)
holdout_messages_df.head()

,filename,filetype,set,messages
0,AES-GCM-SIV-proof/proof/cryptol-specs/AES.cry,cryptol,unsupervised,"[{'role': 'system', 'content': 'Return exactly..."
1,AES-GCM-SIV-proof/proof/cryptol-specs/AES128.cry,cryptol,unsupervised,"[{'role': 'system', 'content': 'Return exactly..."
2,AES-GCM-SIV-proof/proof/cryptol-specs/intrinsi...,cryptol,supervised,"[{'role': 'system', 'content': 'Return exactly..."
3,AES-GCM-SIV-proof/proof/cryptol-specs/AES256.cry,cryptol,unsupervised,"[{'role': 'system', 'content': 'Return exactly..."
4,AES-GCM-SIV-proof/proof/cryptol-specs/TBox.cry,cryptol,unsupervised,"[{'role': 'system', 'content': 'Return exactly..."


In [12]:
holdout_messages_df = holdout_messages_df[holdout_messages_df['set'] == 'holdout'].reset_index(drop=True)

#holdout_messages_df.drop(columns=['set', 'variant'], inplace=True)
holdout_messages_df.head()

,filename,filetype,set,messages
0,aws-lc-verification/cryptol-specs/McEliece_KEM...,cryptol,holdout,"[{'role': 'system', 'content': 'Return exactly..."
1,aws-lc-verification/cryptol-specs/McEliece_KEM...,cryptol,holdout,"[{'role': 'system', 'content': 'Return exactly..."
2,aws-lc-verification/cryptol-specs/Primitive/As...,cryptol,holdout,"[{'role': 'system', 'content': 'Return exactly..."
3,aws-lc-verification/cryptol-specs/Primitive/Sy...,cryptol,holdout,"[{'role': 'system', 'content': 'Return exactly..."
4,aws-lc-verification/cryptol-specs/Primitive/Sy...,cryptol,holdout,"[{'role': 'system', 'content': 'Return exactly..."


In [13]:
print(len(holdout_messages_df))

51


In [14]:
holdout_messages_df.to_json(f"data/holdout_message_format_{VERSION}.jsonl", lines=True, orient='records')

In [15]:
import re
from pathlib import Path

# Regex to capture the first ```cryptol ... ``` block
CRYPTOL_BLOCK_RE = re.compile(
    r"```cryptol\s*(.*?)```",
    re.DOTALL | re.IGNORECASE
)

def extract_cryptol_code(text: str) -> str:
    """
    Given a message string, extract the first ```cryptol ... ``` block.
    """
    m = CRYPTOL_BLOCK_RE.search(text)
    if not m:
        raise ValueError("No ```cryptol ... ``` block found in assistant content.")
    return m.group(1).strip()

def get_assistant_content(messages) -> str:
    """
    messages can be either:
      - a list of { 'role': ..., 'content': ... } dicts (OpenAI-style), or
      - a dict with an 'assistant' key containing the text.
    """
    if isinstance(messages, dict):
        if "assistant" in messages:
            return messages["assistant"]
        raise ValueError("messages dict has no 'assistant' key.")

    # Assume iterable of role/content dicts
    for msg in messages:
        if msg.get("role") == "assistant":
            return msg["content"]
    raise ValueError("No assistant message found in messages list.")

def save_file(filename: str, content: str, encoding: str = "utf-8") -> None:
    """
    Create all missing parent directories and write `content` to `filename`.
    """
    path = Path(filename)
    if path.parent:  # path.parent is '.' for bare filenames
        path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content, encoding=encoding)

def save_cryptol_from_messages(messages, filename: str) -> None:
    """
    Extract ```cryptol ... ``` block from assistant content in `messages`
    and save it to `filename`, rebuilding the directory structure if needed.
    """
    assistant_text = get_assistant_content(messages)
    cryptol_code = extract_cryptol_code(assistant_text)
    save_file(filename, cryptol_code)

In [16]:
for idx, row in holdout_messages_df.iterrows():
    save_cryptol_from_messages(row['messages'], f"evals/{row['filename']}")